In [ ]:
# activate crawler env 1st!
import os
import netCDF4 as nc4
from thredds_crawler.crawl import Crawl

In [ ]:
catalog = 'https://gamone.whoi.edu/thredds/silt/usgs/Projects/stellwagen/Data/catalog.xml'

# these are the 3 keys in the global atts that we're checking
required_attributes = ('MOORING', 'INST_TYPE', 'serial_number')

dirlist = ('ARGO_MERCHANT', 'BARNEGAT', 'BARNEGAT_2014', 'BUZZ_BAY', 'BW_2011')
#dirlist=('CALSCTWET','CAMP','CAPE_COD_BAY','CC_MISC','CHANDELEUR_13','CHINCOTEAGUE')

In [ ]:
c = Crawl(catalog, select=[
    '.*{}.*'.format(x) for x in dirlist])
urls = [s.get("url") for d in c.datasets for s in d.services if s.get("service").lower() == "opendap"]

print("Found {} datasets".format(len(urls)))

In [ ]:
with open('meta_fail_ab.txt', 'w') as f:
    
    for fn in urls:
        # Just getting a prettier representation of the dataset
        spec_file = '/'.join(fn.rsplit('/', maxsplit=2)[1:])
        all_atts = {}
        try:
            with nc4.Dataset(fn) as fnc:
                for k in required_attributes:
                    # .ncattrs returns a list of global attributes
                    if k not in fnc.ncattrs():
                        msg = k + ' not found in ' + spec_file
                        print(msg)
                        f.write(msg)

        except OSError:
            print('failed trying to read ' + spec_file)